Author: Carlos Roberto de Melo

Date: 05/27/2020

Obj: Perform the modeling of spectrum based on pPXF method by Cappellari,and obtein the kinematics./Realizar o modelamento de espectros baeado no código pPXF de Cappellari, e obter a cinemática.

In [1]:
import glob
from os import path

from astropy.io import fits
from scipy import ndimage
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import matplotlib.pyplot as plt

data_folder = "/home/carlos/Documents/GitHub/Master-Degree/SDP81/Dynamics/pPXF (4800-5500)/"

In [2]:
#Directory of stellar library/
stellar_lib = '/home/carlos/Downloads/Bibliotecas Estelares/BINTABLE'
Indo_all    = glob.glob(stellar_lib + '/*.fits')    #All templates
Indo_GK     = []

#Selecting only G and K stars
for i in range(len(Indo_all)):
    hdu = fits.open(Indo_all[i])
    h3 = hdu[1].header
    try:
        if h3['SPTYPE'][0] == "K" or h3['SPTYPE'][0] == "k" or h3['SPTYPE'][0] == "G" or h3['SPTYPE'][0] == "g":
            Indo_GK.append(Indo_all[i])
    except:
        continue

In [3]:
No_Gaps = []

#Selecting templates with no gaps and entiry wavelengt coverage
for i in range(len(Indo_GK)):
    hdu = fits.open(Indo_GK[i])
    h3 = hdu[1].header
    try:
        gap = h3['GAPS']
    except:
        if float(h3["COVERAGE"][7:]) == 9469.0:
            No_Gaps.append(Indo_GK[i])
        continue

In [5]:
ppxf_dir = '/home/carlos/Desktop/ppxf' #Directory of stellar library/Diretória da biblioteca estelar


#stellar_lib = '/home/carlos/Downloads/Bibliotecas Estelares/BINTABLE'
#Indo = glob.glob(stellar_lib + '/*.fits')
FWHM_tem = 1.35


for i in range(8): #Number of bins
     #READING THE SPECTRUM/LENDO UM ESPECTRO
    file = (data_folder+'Data/Annealing_Bin03/Full/Binned_Spec/Bin%d.fits'%i) #Path and name of the spectrum/Caminho e nome do arquivo do espectro
    hdu = fits.open(file) #Opening the .fits file/Abrindo o arquivo.fits
    gal_lin = hdu[1].data #Read the data/Lendo os dados
    h1 = hdu[1].header    #Read the header/Lendo o header dos dados
    
    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO
    var = hdu[2].data     #Read the variance/Lendo a variância
    h_var = hdu[2].header #Read the variance header/Lendo o header da variância
    
    #LOG-REBIN THE SPECTRUM AND VARIANCE/LOGARITMIZANDO O ESPECTRO E A VARIÂNCIA
        #SPECTRUM/ESPECTRO
    FWHM_gal = 2.71  #FWHM of telescope in Angstrons/FWHM do telescópio em Angstrons
    lambRange1 = h1['CRVAL1'] + np.array([0.,h1['CDELT1']*(h1['NAXIS1']-1)]) #Generating the wavelengths/Está linha gera os comprimentos de onda
                                #pois a variável gal_lin armazena em um array apenas as intensidades, mas não os comprimentos de onda
        
    #De-redshift
    z = 0.299 # Initial estimate of the galaxy redshift
    lambRange1 = lambRange1/(1+z) # Compute approximate restframe wavelength range
    FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
    #Depois do passo acima, a galáxia passa a estar no restframe, logo é preeciso setar
    z = 0.
    
    
    galaxy, logLam1, velscale = util.log_rebin(lambRange1, gal_lin) #Log-rebin the spectrum/Logaritimizando o espectro da galáxia
    median_galaxy = np.median(galaxy) #Mean of spectrum for normalization/Média do espectro, pois vamos normalizá-lo pela média
    galaxy = galaxy/median_galaxy        #Normalizing the spectrum/Normalizando o espectro pela média
    lam = np.exp(logLam1)                #Recovering the wavelength in Ansgstrons/Recuperando o comprimento de onda em unidades do SI
        
    #VARIANCE/VARIÂNCIA
    lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])
    erro = np.sqrt(var) #1-sigma error from variance/1-sigma erro da variância
    erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
    noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro        
        
    #READING ONE TEMPLATE FOR DETERMINE THE LENGHT/LENDO UM TEMPLATE PARA SABER O TAMANHO DO VETOR FINAL
        #The ideia is make a empty array to after complete with the informations about the templates of the stellar libary/
                    #A ideia é crirar um array vazio para que mais tarde seja preenchido com as informações de todos os templetes.

    
    
    
    velscale_ratio = 2
    hdu = fits.open(No_Gaps[0])
    data = hdu[1].data
    ssp = data[0][1]
    h2 = hdu[0].header
    
    #Log_rebin TEMPLATE/LOGARITMIZANDO O TEMPLATE
    lamRange2 = np.array([data[0][0][0], data[0][0][len(data[0][0]) - 1]]) 
    sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
    templates = np.empty((sspNew.size, len(No_Gaps)))  #Array com o tamanho necessário

    #FINAL FWHM FOR GALAXY AND TEMPLATE CONVOLUTION/CALCULANDO A FWHM FINAL PARA REALIZAR A CONVOLUÇÃO DO TEMPLATE E GALÁXIA
    FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
    sigma = FWHM_dif/2.355/0.4 #New sigma PSF/Novo sigma da PSF.
    
    #READING ALL TEMPLATES/LENDO TODOS OS TEMPLATES E JÁ LOGARITMIZANDO
    for j, file in enumerate(No_Gaps):
        hdu = fits.open(file)
        data = hdu[1].data
        ssp = data[0][1]
        ssp = ndimage.gaussian_filter1d(ssp, sigma) #Realizando a convolução
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates[:, j] = sspNew/np.median(sspNew)
    
    

    #ADICIONANDO UM SHIFT PARA COINCIDIR OS COMPRIMENTOS DE ONDA DO ESPECTRO E DO TEMPLATE
        #Artificial shift for the galaxy and templates wavelenght may coincide.    
        #Nem sempre os comprimentos de onda do templete e do espectro coincidem, por esse motivo é dado um 
        #shift artificial ao templete, mas que após o ajuste é retirado.
    c = 299792.458 #Speed of light/Velocidade da Luz
    dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c #Shift
    goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)  #Mask bad emission lines/Máscaras para linhas de emissão
    vel = c*np.log(1 + z)
    start = [vel, 200.] #Inicial guess/Chute inicial
    
    #START OF FIT/INÍCIO DO AJUSTE
       #For each spectrum the modeling is done. After the best solution is printed 
       #(velocity, velocity dispersion, another moments). Then, a table is write with the information about the
       #pixel position, velocity, velocity dispersion, chi²/DOF and 1-sigma error. The table is saved with name
       #'Output_Ppxf.txt'.  
        
       #Para cada espectro ele realiza o ajuste e printa abaixo o nome do px em que foi realizado o ajuste,
       #bem como as infomações do melhor ajuste (qui², Velocidade, Dispersão, Demais momentos). Então cria uma
       #tabela com as informações necessárias e a salva com o nome 'Output_Ppxf.txt'.
    pp = ppxf(templates, galaxy, noise, velscale, start,
                lam = lam, plot=True, moments=2,
                  degree=3,vsyst=dv, goodpixels=goodPixels, velscale_ratio=velscale_ratio)
        #For more informations about the method, visit the paper of Cappellari avaible in README
        #Para mais informações sobre o método, visite o paper de Cappellari disponível no README
        
        #Plot and save the best model/Plota e salva o melhor modelo
    pp.plot() #Plot the fit/Plota o fit
    plt.legend(["$\\chi^2/DOF$=%.3f"%pp.chi2])
    plt.savefig(data_folder+'Data/Annealing_Bin03/Full/Spec_Fit/Bin%d.png'%i,format='png', dpi=600)
    plt.clf()
    
    print(i)
    print('Vel :%f.\nDisp:%f.'%(pp.sol[0],pp.sol[1]))
    print('Best-fitting redshift z:', (0.299 + 1)*(1 + pp.sol[0]/c) - 1)
    error = pp.error*np.sqrt(pp.chi2) #Formal Error. dV = error[0], dsigma = error[1]
    #ESCREVENDO UMA TABELA COM OS DADOS DE Bin  VELOCIDADE(KM/S) DISPERSÃO(KM/S) chi2 dV dsigma
    newrow = [np.int(i), pp.sol[0], pp.sol[1], pp.chi2, error[0], error[1]]
    if i == 0:
        table = np.array([np.int(i), pp.sol[0], pp.sol[1],pp.chi2, error[0], error[1]])
    else:
        table = np.vstack([table,newrow])
    print('\n####################\n')

 Best Fit:       Vel     sigma
 comp.  0:       138       244
chi2/DOF: 1.136
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  8 / 102
0
Vel :138.032412.
Disp:243.536025.
Best-fitting redshift z: 0.2995980941091665

####################

 Best Fit:       Vel     sigma
 comp.  0:       142       231
chi2/DOF: 1.359
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  7 / 102
1
Vel :141.875793.
Disp:230.924813.
Best-fitting redshift z: 0.2996147474684836

####################

 Best Fit:       Vel     sigma
 comp.  0:       140       228
chi2/DOF: 1.411
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  7 / 102
2
Vel :139.887879.
Disp:228.358219.
Best-fitting redshift z: 0.2996061338415643

####################

 Best Fit:       Vel     sigma
 comp.  0:       150       217
chi2/DOF: 1.434
method = capfit; Jac calls: 5; Fun

<Figure size 432x288 with 0 Axes>

In [6]:
np.savetxt('Output_Ppxf_IndoTemplate_Full.txt',table, header= 'Bin vel(km/s) disp(km/s) chi2/DOF dV dsigma', fmt='%i %i %.18e %.18e %.18e %.18e',delimiter=' ') 